In [ ]:
# Install Kaggle library
!pip install -q kaggle

In [ ]:
from google.colab import files
#upload the kaggle.json file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
# make a diectoryin which kajggle.json is stored
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
#download the dataset into the colab
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

100% 25.7M/25.7M [00:00<00:00, 18.0MB/s]



In [ ]:
#unzip the data
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, Bidirectional,Flatten,Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def remove_special_chars(tweets):  # it unrolls the hashtags to normal words
    for remove in map(lambda r: re.compile(re.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                     "@", "%", "^", "*", "(", ")", "{", "}",
                                                                     "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                     "!", "?", ".", "'",
                                                                     "--", "---", "#"]):
        tweets.replace(remove, "", inplace=True)
    return tweets

def remove_tags(text):
    return re.compile(r'<[^>]+>').sub('', text)
def remove_num(text):
    return ''.join(re.sub(r"([0-9]+)","",text))

In [ ]:
data = pd.read_csv('/content/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.review=data.review.apply(lambda x : remove_tags(x))
data.review=data.review.apply(lambda x : remove_num(x))
remove_special_chars(data.review)

data.head(14)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive
5,Probably my alltime favorite movie a story of ...,positive
6,I sure would like to see a resurrection of a u...,positive
7,This show was an amazing fresh innovative ide...,negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [ ]:
tokenizer = Tokenizer(num_words=5000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")
tokenizer.fit_on_texts(data['review'])
X = tokenizer.texts_to_sequences(data['review'])
X = pad_sequences(X,maxlen=500)
Y = data['sentiment']
vocab_size = len(tokenizer.word_index) + 1
# We can then create our train and test sets:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 24)

In [ ]:
import pickle
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
tokenizer

In [ ]:
from sklearn.preprocessing import LabelEncoder
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

ytrain,ytest = prepare_targets(Y_train,Y_test)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=500))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 50)           10834550  
_________________________________________________________________
bidirectional_5 (Bidirection (None, 256)               183296    
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 11,018,103
Trainable params: 11,018,103
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [ ]:
history=model.fit(X_train, ytrain,
          batch_size=128,
          epochs=20,
          validation_data=[X_test, ytest],
          callbacks=[es])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 35000 samples, validate on 15000 samples
Epoch 1/20
35000/35000 [==============================] - 717s 20ms/step - loss: 0.4934 - accuracy: 0.7585 - val_loss: 0.3600 - val_accuracy: 0.8495
Epoch 2/20
35000/35000 [==============================] - 716s 20ms/step - loss: 0.2942 - accuracy: 0.8833 - val_loss: 0.3040 - val_accuracy: 0.8749
Epoch 3/20
35000/35000 [==============================] - 710s 20ms/step - loss: 0.2559 - accuracy: 0.8996 - val_loss: 0.2859 - val_accuracy: 0.8843
Epoch 4/20
35000/35000 [==============================] - 711s 20ms/step - loss: 0.2258 - accuracy: 0.9116 - val_loss: 0.3512 - val_accuracy: 0.8761
Epoch 5/20
35000/35000 [==============================] - 713s 20ms/step - loss: 0.2217 - accuracy: 0.9139 - val_loss: 0.3043 - val_accuracy: 0.8786
Epoch 6/20
35000/35000 [==============================] - 709s 20ms/step - loss: 0.2423 - accuracy: 0.9031 - val_loss: 0.3268 - val_accuracy: 0.8787
Epoch 7/20
35000/35000 [==============================] 

In [ ]:
model.save('movie_sent.h5')

In [ ]:
string11='''Between the Lovecraftian overtones and Liberato's performance, 
The Beach House offers up beautifully shot terror and will make you think before opening your door.'''
x_1=tokenizer.texts_to_sequences([string11])
x_1 = pad_sequences(x_1,maxlen=500)
model.predict(x_1)

array([[0.9992687]], dtype=float32)